In [1]:
import os
import re
train_dir = 'C:/Users/25405/Desktop/KG/train'

In [ ]:
def get_categories(dir):
    """
    返回实体类别字典
    """
    categories = {}
    files = os.listdir(dir)
    files = list(set([file.split('.')[0] for file in files]))
    for file in files:
        path = os.path.join(dir, file + '.ann')
        with open(path, 'r', encoding = 'utf8') as f:
            for line in f.readlines():
                category_name = line.split('\t')[1].split(' ')[0]
                if category_name in categories:
                    categories[category_name] += 1
                else:
                    categories[category_name] = 1    
    return categories

def get_labelencoder(categories):
    """
    得到类别标签和下标映射
    """
    categories = sorted(categories.items(), key = lambda x : x[1], reverse = True)
    categories = [x[0] for x in categories]
    label = []
    label.append('O')
    for category in categories:
        label.append('B-' + category)
        label.append('I-' + category)
    label2id = {label[i] : i for i in range(len(label))}
    return label,label2id

def ischinese(char):
    if '\u4e00' <= char <= '\u9fff':
        return True
    return False

def split_text(text):
    split_index = []
    pattern1 = '。|\.|\?'
    #pattern1 = '。|，|,|;|；|\.|\?'
    
    for m in re.finditer(pattern1, text):
        idx = m.span()[0]
        if text[idx - 1].isdigit() and text[idx + 1].isdigit():
            continue
        if text[idx - 1].islower() and text[idx + 1].isdigit():
            continue
        if text[idx - 1].isupper() and text[idx + 1].isdigit():
            continue
        if text[idx + 1] in set('.。;；,，'):
            continue
        if text[idx - 1].isspace() and text[idx - 2].isspace() and text[idx - 3] in set('.。;；,，'):
            continue
        split_index.append(idx + 1)    # 以上情况不满足，切分句子
        
#     pattern2 = '\([一二三四五六七八九零十]\)|[一二三四五六七八九零十]、|'
#     for m in re.finditer(pattern2, text):
#         idx = m.span()[0]
#         split_index.append(idx)
        
    pattern3 = '\d\.'
    for m in re.finditer(pattern3, text):
        idx = m.span()[0]
        if ischinese(text[idx + 2]):
            split_index.append(idx)
    
    
    for m in re.finditer('\(\d\)|\（\d\）',text):
        idx = m.span()[0]
        split_index.append(idx)
    
    split_index = list(sorted(set([0, len(text)] + split_index)))
    
    other_index = []
    for i in range(len(split_index) - 1):
        begin = split_index[i]
        end = split_index[i + 1]
        if text[begin] in '一二三四五六七八九十' or \
                (text[begin] == '(' and text[begin + 1] in '一二三四五六七八九零十'):
            for j in range(begin, end):
                if text[j] == '\n':
                    other_index.append(j + 1)
    split_index += other_index
    split_index = list(sorted(set([0, len(text)] + split_index)))

    other_index = []
    for i in range(len(split_index) - 1):    # 拆分长句子
        b = split_index[i]
        e = split_index[i + 1]
        other_index.append(b)
        if e - b > 150:
            for j in range(b, e):
                if (j + 1 - other_index[-1]) >  15:       # 保证句子长度大于15
                    if text[j] == '\n':
                        other_index.append(j + 1)
                    if text[j] == ' ' and text[j - 1].isnumeric() and text[j + 1].isnumeric():
                        other_index.append(j + 1)
    split_index += other_index
    split_index = list(sorted(set([0, len(text)] + split_index)))

    for i in range(1, len(split_index) - 1):     # 去掉全部是空格的句子
        idx = split_index[i]
        while idx > split_index[i - 1] - 1 and text[idx - 1].isspace():
            idx -= 1
        split_index[i] = idx
    split_index = list(sorted(set([0, len(text)] + split_index)))
    
    # 处理短句
    temp_idx = []
    i = 0
    while i < len(split_index) - 1:
        b = split_index[i]
        e = split_index[i + 1]

        num_ch = 0
        num_en = 0
        if e - b < 15:
            for ch in text[b : e]:
                if ischinese(ch):
                    num_ch += 1
                elif ch.islower() or ch.isupper():
                    num_en += 1
                if num_ch + 0.5 * num_en > 10:
                    temp_idx.append(b)
                    i += 1
                    break
            if num_ch + 0.5 * num_en <= 10:
                temp_idx.append(b)
                i += 2
        else:
            temp_idx.append(b)
            i += 1
    split_index = list(sorted(set([0, len(text)] + temp_idx)))
    
    result = []
    for i in range(len(split_index) - 1):
        result.append(text[split_index[i] : split_index[i + 1]])

    #检查
    s = ''
    for r in result:
        s += r
    assert  len(s) == len(text)
    return result
    
    
if __name__ == '__main__':
    
#     print(get_labelencoder(get_categories(train_dir)))
    files = os.listdir(train_dir)
    files = list(set([file.split('.')[0] for file in files]))

path = os.path.join(train_dir, files[1] + '.txt')
with open(path, 'r', encoding = 'utf8') as f:
    text = f.read()
    print(split_text(text),'\n', text)